In [31]:
# 1. 라이브러리 임포트
import os
import yaml
import torch
from ultralytics import YOLO
from roboflow import Roboflow
import pandas as pd
import matplotlib.pyplot as plt

In [32]:
# 2. GPU 또는 CPU 설정
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")
# 학습 파라미터 설정
EPOCHS = 20  # 에포크 수 (필요에 따라 조절)
BATCH_SIZE = 16 # 배치 사이즈

Using device: cuda


In [33]:
from roboflow import Roboflow
rf = Roboflow(api_key="NjIXpou4o4gsuGClT8hI")
project = rf.workspace("test-l2t0m").project("acne04-bn4vo-xdhi8")
version = project.version(1)
dataset = version.download("yolov11")
rf = Roboflow(api_key="NjIXpou4o4gsuGClT8hI")
project = rf.workspace("test-l2t0m").project("acne-gvhhe-bcjln")
version = project.version(1)
dataset = version.download("yolov11")
                                
                
                
# 다운로드된 데이터셋 경로 확인
DATASET_DIR = dataset.location
print(f"Dataset downloaded to: {DATASET_DIR}")                

loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...
Dataset downloaded to: c:\Users\Admin\work space\2nd\Acne-1


In [34]:
# 4. data.yaml 생성 (파일이 누락된 경우를 위한 코드)

try:
    # 클래스 이름과 개수를 Roboflow 프로젝트에서 직접 가져오기
    CLASS_NAMES = list(project.classes.keys())
    NUM_CLASSES = len(CLASS_NAMES)

    print(f"프로젝트에서 {NUM_CLASSES}개의 클래스를 확인했습니다:")
    print(CLASS_NAMES)

    # data.yaml 파일 경로 설정
    yaml_path = os.path.join(DATASET_DIR, 'data.yaml')

    # YAML 파일에 쓸 내용 구성
    data_to_write = {
        'train': os.path.join(DATASET_DIR, 'train/images'),
        'val': os.path.join(DATASET_DIR, 'valid/images'), # Roboflow는 보통 'valid'를 사용
        'test': os.path.join(DATASET_DIR, 'test/images'),
        'nc': NUM_CLASSES,
        'names': CLASS_NAMES
    }

    # 파일 작성
    with open(yaml_path, 'w') as f:
        yaml.dump(data_to_write, f)

    print(f"\n성공: '{yaml_path}'에 data.yaml 파일을 생성했습니다.")

except Exception as e:
    print(f"data.yaml 생성 중 오류 발생: {e}")

프로젝트에서 5개의 클래스를 확인했습니다:
['cyst', 'nodule', 'comedone', 'papule', 'pustule']

성공: 'c:\Users\Admin\work space\2nd\Acne-1\data.yaml'에 data.yaml 파일을 생성했습니다.


In [35]:
# 4. 데이터셋의 클래스 정보 자동 로드 및 YAML 파일 재설정 (수정된 버전)

# Roboflow가 제공한 data.yaml 파일을 읽어 클래스 정보 추출
original_yaml_path = os.path.join(DATASET_DIR, 'data.yaml')
with open(original_yaml_path, 'r') as f:
    data_yaml = yaml.safe_load(f)

# 클래스 이름과 개수를 자동으로 설정
CLASS_NAMES = data_yaml['names']
NUM_CLASSES = data_yaml['nc']

print(f"Detected {NUM_CLASSES} classes:")
print(CLASS_NAMES)

# --- ⭐️ 오류 수정 핵심 부분 ⭐️ ---
# 'valid' 또는 'val' 폴더 중 실제 존재하는 폴더 이름을 확인합니다.
val_folder_name = ''
if os.path.isdir(os.path.join(DATASET_DIR, 'valid')):
    val_folder_name = 'valid'
    print("\n'valid' 폴더를 검증 데이터셋으로 사용합니다.")
elif os.path.isdir(os.path.join(DATASET_DIR, 'val')):
    val_folder_name = 'val'
    print("\n'val' 폴더를 검증 데이터셋으로 사용합니다.")
else:
    raise FileNotFoundError(f"'{DATASET_DIR}' 경로에서 'valid' 또는 'val' 폴더를 찾을 수 없습니다.")
# --- ⭐️ 오류 수정 핵심 부분 끝 ⭐️ ---


# 5. 학습을 위한 새로운 data.yaml 파일 생성 (절대 경로 사용)
# YOLO 학습 시 경로 문제를 방지하기 위해 절대 경로로 설정
new_data_yaml = {
    'path': os.path.abspath(DATASET_DIR), # 절대 경로로 수정
    'train': 'train/images',
    'val': os.path.join(val_folder_name, 'images'), # 자동으로 감지된 폴더 이름으로 경로 설정
    'test': 'test/images',
    'nc': NUM_CLASSES,
    'names': CLASS_NAMES
}

# 현재 작업 폴더에 새로운 YAML 파일 저장
new_yaml_path = os.path.join(os.getcwd(), 'acne_lesion_data.yaml')
with open(new_yaml_path, 'w') as f:
    yaml.dump(new_data_yaml, f)

print(f"\nNew YAML file created at: {new_yaml_path}")

# 생성된 YAML 내용 확인 (디버깅용)
print("\n--- 생성된 YAML 파일 내용 ---")
print(yaml.dump(new_data_yaml))
print("----------------------------")

Detected 5 classes:
['cyst', 'nodule', 'comedone', 'papule', 'pustule']

'valid' 폴더를 검증 데이터셋으로 사용합니다.

New YAML file created at: c:\Users\Admin\work space\2nd\acne_lesion_data.yaml

--- 생성된 YAML 파일 내용 ---
names:
- cyst
- nodule
- comedone
- papule
- pustule
nc: 5
path: c:\Users\Admin\work space\2nd\Acne-1
test: test/images
train: train/images
val: valid\images

----------------------------


In [36]:
# 5. 생성된 data.yaml을 바탕으로 학습 설정 계속 진행

original_yaml_path = os.path.join(DATASET_DIR, 'data.yaml')
with open(original_yaml_path, 'r') as f:
    data_yaml = yaml.safe_load(f)

CLASS_NAMES = data_yaml['names']
NUM_CLASSES = data_yaml['nc']

# 'valid' 또는 'val' 폴더 확인
val_folder_name = ''
if os.path.isdir(os.path.join(DATASET_DIR, 'valid')):
    val_folder_name = 'valid'
elif os.path.isdir(os.path.join(DATASET_DIR, 'val')):
    val_folder_name = 'val'
else:
    raise FileNotFoundError(f"'{DATASET_DIR}' 경로에서 'valid' 또는 'val' 폴더를 찾을 수 없습니다.")

# 학습에 사용할 최종 YAML 파일 생성
new_data_yaml = {
    'path': os.path.abspath(DATASET_DIR),
    'train': 'train/images',
    'val': os.path.join(val_folder_name, 'images'),
    'test': 'test/images',
    'nc': NUM_CLASSES,
    'names': CLASS_NAMES
}

new_yaml_path = os.path.join(os.getcwd(), 'acne_lesion_data.yaml')
with open(new_yaml_path, 'w') as f:
    yaml.dump(new_data_yaml, f)

print(f"학습 준비 완료. 최종 설정 파일: '{new_yaml_path}'")

학습 준비 완료. 최종 설정 파일: 'c:\Users\Admin\work space\2nd\acne_lesion_data.yaml'


In [37]:
# 6. 모델 정의: YOLO11 전이 학습
model = YOLO('yolo11n.pt')  # YOLO11 나노 버전 사전 훈련 모델 로드
model.to(DEVICE)
print("YOLO model loaded successfully.")

YOLO model loaded successfully.


In [38]:
# 7. 모델 학습 실행
results = model.train(
    data=new_yaml_path,
    epochs=EPOCHS,
    imgsz=640,  # 이미지 사이즈 (YOLO는 보통 640x640 사용)
    batch=BATCH_SIZE,
    device=DEVICE,
    project='acne_lesion_detection', # 결과 저장 폴더
    name='yolo11n_lesion_detection_run', # 실행 이름
    patience=10, # 조기 종료 (10 에포크 동안 성능 향상이 없으면 중단)
    exist_ok=True
)

New https://pypi.org/project/ultralytics/8.3.201 available  Update with 'pip install -U ultralytics'
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\Users\Admin\work space\2nd\acne_lesion_data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_lesion_detection_run, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overl

In [39]:
# 8. 학습된 모델 평가 (테스트 데이터셋 사용)
print("\\nEvaluating model on the test set...")
# 최상의 가중치(best.pt)는 results.save_dir 경로에 저장됩니다.
best_model_path = os.path.join(results.save_dir, 'weights/best.pt')

# 만약 best.pt 경로를 찾을 수 없다면, 새로 모델을 로드합니다.
if not os.path.exists(best_model_path):
    # 직접 경로를 지정해야 할 경우
    # 예: best_model_path = 'acne_lesion_detection/yolo11n_lesion_detection_run/weights/best.pt'
    raise FileNotFoundError("Could not find best.pt. Please check the training output path.")

best_model = YOLO(best_model_path)

# 테스트 데이터셋으로 검증
metrics = best_model.val(
    data=new_yaml_path,
    split='test',
    project='acne_lesion_detection',
    name='test_results'
)

print("\\nTest Metrics:")
print(f"mAP50-95: {metrics.box.map}")
print(f"mAP50: {metrics.box.map50}")

\nEvaluating model on the test set...
Ultralytics 8.3.200  Python-3.13.7 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 100 layers, 2,583,127 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 611.9541.6 MB/s, size: 574.3 KB)
val: Scanning C:\Users\Admin\work space\2nd\Acne-1\test\labels.cache... 92 images, 0 backgrounds, 22 corrupt: 100% ━━━━━━━━━━━━ 92/92 193.7Kit/s 0.0s
val: C:\Users\Admin\work space\2nd\Acne-1\test\images\01F3MMX2RXFSJV79X3QMF6XXW8_jpeg.rf.fbb8e090b35245590e3e808f6c1f38eb.jpg: ignoring corrupt image/label: Label class 5 exceeds dataset class count 5. Possible class labels are 0-4
val: C:\Users\Admin\work space\2nd\Acne-1\test\images\01F3MMY1W7BKARQ7SMST6FA3WF_jpeg.rf.c3b389893c9588d362ee5fa9b66490fd.jpg: ignoring corrupt image/label: Label class 5 exceeds dataset class count 5. Possible class labels are 0-4
val: C:\Users\Admin\work space\2nd\Acne-1\test\images\01F3MMY6511W62QXS9V7R450

In [40]:
# 9. 학습 결과 시각화
results_csv_path = os.path.join(results.save_dir, 'results.csv')
if os.path.exists(results_csv_path):
    results_df = pd.read_csv(results_csv_path)
    
    # 공백 제거
    results_df.columns = results_df.columns.str.strip()
    
    plt.figure(figsize=(15, 5))
    
    # Loss 시각화
    plt.subplot(1, 2, 1)
    plt.plot(results_df['epoch'], results_df['train/box_loss'], label='Train Box Loss')
    plt.plot(results_df['epoch'], results_df['val/box_loss'], label='Validation Box Loss')
    plt.plot(results_df['epoch'], results_df['train/cls_loss'], label='Train Class Loss')
    plt.plot(results_df['epoch'], results_df['val/cls_loss'], label='Validation Class Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    # Metrics 시각화
    plt.subplot(1, 2, 2)
    plt.plot(results_df['epoch'], results_df['metrics/mAP50(B)'], label='mAP@0.5')
    plt.plot(results_df['epoch'], results_df['metrics/mAP50-95(B)'], label='mAP@0.5-0.95')
    plt.title('Validation mAP')
    plt.xlabel('Epoch')
    plt.ylabel('mAP')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
else:
    print("Results CSV file not found. Cannot visualize results.")

<Figure size 1500x500 with 2 Axes>